In [1]:
# Imports
import os, cv2, glob
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass
from skimage.filters import gaussian
from skimage.util import random_noise
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Initialization

# making input and output folders
os.makedirs(f"/content/drive/MyDrive/Input", exist_ok=True)
ROOT = "/content/drive/MyDrive/Input"

os.makedirs(f"{ROOT}/in_pics", exist_ok=True)
os.makedirs(f"{ROOT}/in_masks", exist_ok=True)

os.makedirs(f"{ROOT}/out_pics", exist_ok=True)
os.makedirs(f"{ROOT}/out_masks", exist_ok=True)

# change depending on type of input images
in_pics_type = ".png"
in_masks_type = ".png"

# defining necessary variables
in_pics_path = f"{ROOT}/in_pics" # type: png
in_masks_path = f"{ROOT}/in_masks" # type: png

out_pics_path = f"{ROOT}/out_pics" # type: jpg
out_masks_path = f"{ROOT}/out_masks" # type: tif

In [10]:
# before running this cell, place input images in
# Input/in_pics and input masks in Input/in_masks,
# where images and corresponding masks have same name
pics = os.listdir(in_pics_path)
masks = os.listdir(in_pics_path)

# lengths should be the same
print(f"{len(pics)} input pics")
print(f"{len(masks)} input masks")

1 input pics
1 input masks


In [8]:
# Defining functions needed for augmentation

# normalization function for augmentation
def histNormal(image, min, max):
  image = np.asarray(image).astype(float)
  image = (image - min) / (max - min) * 255
  return np.clip(image, 0, 255)

# image saving
def im_save(fname, func, image, mask):
  newName = f"{fname}{func}_pics_{str(a)}.jpg"
  newNameMask = f"{fname}{func}_pics_{str(a)}.tif"
  newPath = out_pics_path + "/" + newName
  newPathMask = out_masks_path + "/" + newNameMask
  cv2.imwrite(newPath, image)
  mask.save(newPathMask, "TIFF")

In [11]:
# Augmentation

for a in tqdm(range(0, 360, 36)): # a = angle of rotation
  for fname in pics:
    fname = fname.split('.')[0]

    image_path = in_pics_path + "/" + fname + in_pics_type
    mask_path = in_masks_path + "/" + fname + in_masks_type
    image = Image.open(image_path).convert('L')
    mask = Image.open(mask_path).convert('L')

    # crop img
    y, x = center_of_mass(np.asarray(image))
    buff = 75 # change buffer depending on proportion of image that is tumor
    image = image.crop((x - buff, y - buff, x + buff, y + buff))
    mask = mask.crop((x - buff, y - buff, x + buff, y + buff))

    # rotate
    image = image.rotate(a,resample=Image.BILINEAR)
    mask = mask.rotate(a,resample=Image.BILINEAR)
    im_save(fname, "rotated", np.asarray(image), mask)

    # histogram normalization
    histImage = histNormal(image,40,210)
    histImage = Image.fromarray(histImage)
    histImage = histImage.convert('L')
    im_save(fname, "histNorm", np.asarray(histImage), mask)

    # gaussian blur
    gaussImage = gaussian(np.asarray(image), sigma=7)
    gaussImage = cv2.normalize(gaussImage, None, 0, 255, cv2.NORM_MINMAX)
    im_save(fname, "gauss", gaussImage, mask)

    # noise
    noiseImage = random_noise(np.asarray(image), mode = 'gaussian', var=0.01)
    noiseImage = cv2.normalize(noiseImage, None, 0, 255, cv2.NORM_MINMAX)
    im_save(fname, "noise", noiseImage, mask)

100%|██████████| 10/10 [00:00<00:00, 14.61it/s]
